# Antimatter rocket

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lookbusy1344/Relativity/blob/main/Python/Antimatter%20rocket.ipynb)

This repository is primarily focused on the purely kinematic equations of Special Relativity. However, it also includes some basic functions for an antimatter rocket, in both Python and Typescript. These are used in the interactive web calculator:

https://lookbusy1344.github.io/Relativity/

The Python propulsion functions are in https://github.com/lookbusy1344/Relativity/blob/main/Python/propulsion.py

## The ultimate fuel - antimatter

Antimatter is the most energy-dense fuel possible, with a specific energy of approximately **9x10^16 J/kg** (90 PJ/kg) from proton-antiproton annihilation. This is about **9 billion times** the energy density of chemical fuels like hydrogen-oxygen, and about **3000 times** that of nuclear fission or fusion. To release this energy, matter and antimatter just has to come into contact, annihilating each other and producing high-energy photons (gamma rays) and other particles.

The entire rest mass of the fuel is converted into energy, eg for half a gram of antimatter annihilating with half a gram of matter:

$$E = mc^2 = 0.001 \text{ kg} \cdot (3 \times 10^8)^2$$

$$E \approx 9 \times 10^{13}\text{ J}$$

### TNT Equivalent

$$\text{1 megaton TNT} = 4.184 \times 10^{15} \text{ J}$$

$$\text{megatons} = \frac{9 \times 10^{13}}{4.184 \times 10^{15}}$$

$$\approx 0.0215\text{ megatons}$$
$$\text{1g matter/antimatter} \approx 21.5 \text{ kilotons}$$

So 1 gram of matter/antimatter annihilation releases energy equivalent to **21.5 kilotons** of TNT, almost exactly the yield of the Nagasaki bomb. 1 gram of a dense material like lead (or anti-lead!) has a volume of just 88 mm^3, or a sphere with a diameter of about half a cm.

Roughly the size of a pea, for an explosion large enough to destroy a city! Less violently, that pea could power 9,000-10,000 homes for a year.

## Two kinds of antimatter rocket

There are two basic kinds of antimatter rocket:

1. **Photonic rocket**: using the energy from matter-antimatter annihilation to produce photons (gamma rays) which are emitted out the back of the rocket to produce thrust. This is the simplest kind of antimatter rocket, but is unrealistic because it is very difficult to collimate the gamma rays into a beam.

2. **Charged-pion rocket**: using charged pions produced in proton-antiproton annihilation to produce thrust. NASA experimental studies show that approximately **40% of the annihilation energy** appears as charged pion kinetic energy that can be magnetically collimated for thrust. The remaining ~60% goes to neutral pions (which decay to gamma rays), pion rest mass energy, and other particles that cannot be efficiently directed.

## We use the charged-pion rocket equations here

This technology is beyond our current capabilities, but is scientifically plausible. We assume various levels of nozzle efficiency to account for engineering challenges in collimating the charged pions. A nozzle efficiency of 85% with 40% energy in charged pions gives **~34% total system efficiency**, which is probably near the maximum achievable.

**References:**
- NASA antimatter propulsion studies: https://ntrs.nasa.gov/api/citations/20200001904/downloads/20200001904.pdf
- NASA contractor report: https://ntrs.nasa.gov/api/citations/19890018329/downloads/19890018329.pdf
- Experimental data: https://link.springer.com/article/10.1140/epja/s10050-024-01428-x

In [8]:
# Download propulsion.py from your GitHub repo if not present. Important for colab.
import os

if not os.path.exists("relativity_lib.py"):
    !wget -q https://raw.githubusercontent.com/lookbusy1344/Relativity/main/Python/relativity_lib.py

if not os.path.exists("propulsion.py"):
    !wget -q https://raw.githubusercontent.com/lookbusy1344/Relativity/main/Python/propulsion.py

import mpmath as mp
import pandas as pd
import relativity_lib as rl
import propulsion as prop

# Configure relativity_lib for precision calculations
rl.configure(100)

fuel_mass = mp.mpf(1000)  # kg, half is matter, half is antimatter
dry_mass = mp.mpf(500)  # kg, the spacecraft dry mass

t = prop.pion_rocket_accel_time(fuel_mass, dry_mass, nozzle_efficiency=0.85)
years = t / rl.seconds_per_year
print(
    f"Dry mass {float(dry_mass):.0f}, fuel mass {float(fuel_mass):.0f} means {float(years):.2f} years at 1g "
    f"(85% nozzle efficiency, ~34% total system efficiency)"
)
print()

# Calculate propellant fraction for 3.52 years of 1g acceleration at 85% nozzle efficiency
years = rl.ensure("3.52")
secs = rl.seconds_per_year * years
f = prop.pion_rocket_fuel_fraction(secs, None, 0.85)
print(
    f"Propellant mass fraction: {float(f) * 100:.4f}% for {float(years)} years of 1g "
    f"(85% nozzle efficiency, ~34% total system efficiency)"
)

perc = mp.mpf("100")

# Define efficiencies and time periods
nozzle_efficiencies = [0.70, 0.75, 0.80, 0.85]
proper_times_years = [0.01, 0.1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 48]

# Collect data for DataFrame
data = []
for eff in nozzle_efficiencies:
    for years in proper_times_years:
        secs = rl.seconds_per_year * rl.ensure(str(years))
        f = prop.pion_rocket_fuel_fraction(secs, None, eff)
        frac = rl.format_mpf_significant(f * perc, 2, "9")
        data.append([f"{eff:.0%}", years, frac])

# Create DataFrame
df = pd.DataFrame(
    data,
    columns=["Nozzle efficiency", "Proper time (years)", "Propellant fraction (%)"],
)


# Function to highlight first row of each efficiency group
def highlight_efficiency_change(row):
    # Get the current efficiency value
    eff = row["Nozzle efficiency"]
    # Check if this is the first occurrence of this efficiency
    idx = row.name
    if idx == 0 or df.iloc[idx - 1]["Nozzle efficiency"] != eff:
        return ["background-color: #e8f4f8; text-align: right"] * len(row)
    return ["text-align: right"] * len(row)


print()
print(
    "Here we show propellant fractions (how much of the ship's mass needs to be fuel) for various nozzle efficiencies and proper times at 1g acceleration"
)
print("Nozzle efficiencies range from 70% to 85%, and various durations")
print("Rows are highlighted where nozzle efficiency changes")

# Apply styling and display
styled_df = (
    df.style.apply(highlight_efficiency_change, axis=1)
    .format({"Proper time (years)": "{:.2f}"})
    .hide(axis="index")
)
display(styled_df)

Dry mass 500, fuel mass 1000 means 0.34 years at 1g (85% nozzle efficiency, ~34% total system efficiency)

Propellant mass fraction: 99.9988% for 3.52 years of 1g (85% nozzle efficiency, ~34% total system efficiency)

Here we show propellant fractions (how much of the ship's mass needs to be fuel) for various nozzle efficiencies and proper times at 1g acceleration
Nozzle efficiencies range from 70% to 85%, and various durations
Rows are highlighted where nozzle efficiency changes


Nozzle efficiency,Proper time (years),Propellant fraction (%)
70%,0.01,3.84
70%,0.10,32.44
70%,1.00,98.02
70%,2.00,99.960
70%,3.00,99.99922
70%,4.00,99.999984
70%,5.00,99.99999969
70%,6.00,99.9999999939
70%,7.00,99.99999999988
70%,8.00,99.9999999999976


## Crossing the observable universe

48 years of proper time at 1g is sufficient to cross the observable universe, flipping half way to return to the original frame, see [this notebook for details](https://github.com/lookbusy1344/Relativity/blob/main/Python/Universe.ipynb).

However such a journey would require an inconceivable propellant fraction. Even without slowing down to the original frame, the propellant fraction is still enormous. Lets work out the required fuel mass.

In [9]:
dry_mass = mp.mpf("500")  # kg
eff = 0.85  # nozzle efficiency
secs = rl.seconds_per_year * mp.mpf(
    "47.67"
)  # 47.67 proper years for 47 billion light years at 1g
galaxy = mp.mpf("3e42")  # kg, mass of Milky Way

fraction = prop.pion_rocket_fuel_fraction(secs, None, eff)
fuel_mass = dry_mass * (fraction / (1 - fraction))
print()
print(f"To cross the observable universe at 1g with 85% nozzle efficiency")
print(f"Mass fraction for fuel {rl.format_mpf_significant(fraction, 2, '9')}")
# print(f"The fuel mass required is {rl.format_mpf(fuel_mass,0)} kg")
print()
print(f"Ship dry mass of {rl.format_mpf(dry_mass, 0)} kg")
print(f"The fuel mass required is {rl.format_mpf_sci(fuel_mass, 4)} kg")
print(f"Mass of Milky Way is {rl.format_mpf_sci(galaxy)} kg")
print()
print(f"This is {rl.format_mpf(fuel_mass / galaxy, 1)} times the mass of the Milky Way")


To cross the observable universe at 1g with 85% nozzle efficiency
Mass fraction for fuel 0.99999999999999999999999999999999999999999999999999999999999999999986

Ship dry mass of 500 kg
The fuel mass required is 3.7e+69 kg
Mass of Milky Way is 3.0e+42 kg

This is 1,233,464,005,645,591,707,482,156,399.4 times the mass of the Milky Way


## Conclusion

To cross the observable universe in a human lifetime, even with an antimatter rocket, you would need around **1.2 octillion times the mass of the Milky Way** as fuel, for a small 500 kg ship. This is clearly ridiculous, and shows the practical limits of intergalactic travel even with the most advanced propulsion technology we can currently conceive of.

More relativity notebooks are [available here](https://github.com/lookbusy1344/Relativity/blob/main/Python/README.md)

You can read about antimatter rockets in more detail on [Wikipedia](https://en.wikipedia.org/wiki/Antimatter_rocket).